# What is the True Normal Human Body Temperature? 

#### Background

The mean normal body temperature was held to be 37$^{\circ}$C or 98.6$^{\circ}$F for more than 120 years since it was first conceptualized and reported by Carl Wunderlich in a famous 1868 book. But, is this value statistically correct?

<div class="span5 alert alert-info">
<h3>Exercises</h3>

<p>In this exercise, you will analyze a dataset of human body temperatures and employ the concepts of hypothesis testing, confidence intervals, and statistical significance.</p>

<p>Answer the following questions <b>in this notebook below and submit to your Github account</b>.</p> 

<ol>
<li>  Is the distribution of body temperatures normal? 
    <ul>
    <li> Although this is not a requirement for CLT to hold (read CLT carefully), it gives us some peace of mind that the population may also be normally distributed if we assume that this sample is representative of the population.
    </ul>
<li>  Is the sample size large? Are the observations independent?
    <ul>
    <li> Remember that this is a condition for the CLT, and hence the statistical tests we are using, to apply.
    </ul>
<li>  Is the true population mean really 98.6 degrees F?
    <ul>
    <li> Would you use a one-sample or two-sample test? Why?
    <li> In this situation, is it appropriate to use the $t$ or $z$ statistic? 
    <li> Now try using the other test. How is the result be different? Why?
    </ul>
<li>  At what temperature should we consider someone's temperature to be "abnormal"?
    <ul>
    <li> Start by computing the margin of error and confidence interval.
    </ul>
<li>  Is there a significant difference between males and females in normal temperature?
    <ul>
    <li> What test did you use and why?
    <li> Write a story with your conclusion in the context of the original problem.
    </ul>
</ol>

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources

+ Information and data sources: http://www.amstat.org/publications/jse/datasets/normtemp.txt, http://www.amstat.org/publications/jse/jse_data_archive.htm
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

****

In [22]:
import pandas as pd

df = pd.read_csv('data/human_body_temperature.csv')

## 1. Is the distribution of body temperatures normal? 

Yes, human body temperature is a biological process, so we can assume the distribution will be random. Biological processes tend to be influenced by a large number of many random effects which signals normal distribution.


## 2. Is the sample size large? Are the observations independent

In [23]:
sample_size = len(df)
sample_size


130

The sample size is large enough as we have more than 30 samples.
Observations are independent as there is no connection between different samples of body temperature. Which satisfies the [formal definition](https://stats.stackexchange.com/questions/116355/what-does-independent-observations-mean) of independece. "The occurrence of one event doesn't change the probability for another."

## 3. Is the true population mean really 98.6 degrees F?

In [24]:
import numpy
import matplotlib.pyplot as pyplot

%matplotlib inline

In [25]:
# using re-sample function from compstats exercise
def resample(sample, n, replace=True):
    """Generates a new sample by choosing from the original
    sample with replacement.
    """
    new_sample = numpy.random.choice(sample, n, replace)
    return new_sample

# using sample_stat function from compstats exercise
def sample_stat(sample):
    """Computes a sample statistic using the original sample or a
    simulated sample.
    """
    return sample.mean()

# using compute stats function from compstats exercise
def compute_sample_means(sample, n, iters=1000):
    """Simulates many experiments and collects the resulting sample
    statistics.
    """
    stats = [sample_stat(resample(sample, n)) for i in range(iters)]
    return numpy.array(stats)

def t_score(value, mean, std, sample_size):
    signal = value - mean
    noise = std / numpy.sqrt(sample_size - 1)
    return signal / noise


In [26]:
historic_mean = 98.6

# null hypothesis - Historic mean is the true mean
# we calculate probability of null hypothesis being true

# get temperatures from data frame and create numpy array
temperatures = numpy.array(df.temperature)
temperatures.mean(), temperatures.std()

(98.249230769230749, 0.73035777890503795)

In [27]:
historic_t_score = t_score(historic_mean, temperatures.mean(), temperatures.std(), sample_size)
historic_t_score

5.4548232923645195

Looking up the resulting t-score value in the [t-score table](http://www.sjsu.edu/faculty/gerstman/StatPrimer/t-table.pdf) for 129 degrees of freedom (sample size - 1), we can see the historic temperature mean is less than 0.01% likely to be the true mean of the population. So we reject the null hypthosis and we consider the historic mean not to be true mean.

### 3a. Would you use a one-sample or two-sample test? Why?


We have only one group samples so we use one-sample text.

### 3b. In this situation, is it appropriate to use the t or  z statistic?

We have a large sample size (over 30) but we do not know the standard deviation of the population. We have just an estimate. There we use t statistic.

### 3c. Now try using the other test. How is the result be different? Why?

Looking up the resulting value in the [z-score table](http://www.stat.ufl.edu/~athienit/Tables/Ztable.pdf) we see again, that the historic mean is unlikely to be the true temperature mean for the entire population and we reject the null hypothesis.

For samples larger than 30, the result of z tests and t tests start to converge.

## 4. At what temperature should we consider someone's temperature to be "abnormal"?

In [28]:
# resample
sample_means = compute_sample_means(temperatures, 50)

In [29]:
# standard error
std_error = sample_means.std()
std_error

0.10452142438753903

In [36]:
# confidence interval
numpy.percentile(sample_means, [5, 95])

array([ 98.07 ,  98.416])

The temperatures should be consident abnormal below 98.07 and above 98.146 degrees as there's less than 5% chance to get these results just by random.

## 5. Is there a significant difference between males and females in normal temperature?

Using the Cohens'D test as it approviates measure of overlap of the both group distributions' and some intuition
if there's a significant difference

In [31]:
## NULL hypothesis - there isn't a difference between males and females

In [32]:
def CohenEffectSize(group1, group2):
    """Compute Cohen's d.

    group1: Series or NumPy array
    group2: Series or NumPy array

    returns: float
    """
    diff = group1.mean() - group2.mean()

    n1, n2 = len(group1), len(group2)
    var1 = group1.var()
    var2 = group2.var()

    pooled_var = (n1 * var1 + n2 * var2) / (n1 + n2)
    d = diff / numpy.sqrt(pooled_var)
    return d

In [33]:
females = df[df.gender == 'F'].temperature
males = df[df.gender == 'M'].temperature

sample_females = compute_sample_means(females, 50)
sample_males = compute_sample_means(males, 50)

In [34]:
d = CohenEffectSize(sample_females, sample_males)
d

2.7906176326977659

The observed value of Cohen's D is quite high, meaning there's a little overlap in the distribution of male and female temperatures, so we can assume there is a significant difference between male and female temperatures.

## Conclusion

Our analysis concluded that the originally reported historical value of 37 ∘ C (98.6 ∘ F) is not likely to be the true human body temperature.

Aditionally, we've proven that there is a significant difference in body temperatures between males and females. This raise a question whether it's very meaningful to even report the mean human body temperature. Unless you know the gender of person, it is very difficult to asses "abnormality" of ones body temperature.